<a href="https://colab.research.google.com/github/UDICatNCHU/PyTorch-Tutorial/blob/master/Lesson_7_BERT_on_%E5%A4%96%E8%B3%A3%E8%B3%87%E6%96%99%E9%9B%86_(non_verbos).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 確認 GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7aa91c2c-cf39-b858-5c8d-908748c6f8a8)


### Mount 雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### 實作 BERT 情緒分類 



dataset 來源: https://github.com/SophonPlus/ChineseNlpCorpus

#### 下載資料集

In [ ]:
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/waimai
%cd /content/drive/My\ Drive/Colab\ Notebooks/waimai

mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/waimai’: File exists
/content/drive/My Drive/Colab Notebooks/waimai


In [ ]:
!wget -O  waimai_10k_zh_tw.csv "https://www.dropbox.com/s/opundqbrxock8fi/waimai_10k_zh_tw.csv?dl=0"

--2020-11-09 10:17:09--  https://www.dropbox.com/s/opundqbrxock8fi/waimai_10k_zh_tw.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/opundqbrxock8fi/waimai_10k_zh_tw.csv [following]
--2020-11-09 10:17:09--  https://www.dropbox.com/s/raw/opundqbrxock8fi/waimai_10k_zh_tw.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7aa3a168d9a0e5361f8f1316c7.dl.dropboxusercontent.com/cd/0/inline/BC3Xyty-dgpL4kLMCP80nbjGhgiIZgp0MjF6Njow2t26XXdEH55fmW6GCnZBtfnAmczvCxoV_-sp6QoMlX8osSDfzlCTV9l8-e_6sRGz3k_JoF4PbYzmQGUqHKgf_twzakU/file# [following]
--2020-11-09 10:17:10--  https://uc7aa3a168d9a0e5361f8f1316c7.dl.dropboxusercontent.com/cd/0/inline/BC3Xyty-dgpL4kLMCP80nbjGhgiIZgp0MjF6Njow2t26XXdEH55fmW6GCnZBtfnAmczvCxoV_-sp6QoMlX8os

In [ ]:
!ls

'BERT on 外賣資料集.ipynb'   waimai_10k_zh_tw.csv


#### 外賣資料集格式
label | review |
:--- | :--- |
`1`代表正面<br><br>`0`代表負面 |  評論內容 |

In [ ]:
import pandas as pd
pd_all = pd.read_csv('./waimai_10k_zh_tw.csv')

print('評論總數：%d' % pd_all.shape[0])
print('正面評論：%d' % pd_all[pd_all.label==1].shape[0])
print('負面評論：%d' % pd_all[pd_all.label==0].shape[0])

評論總數：11987
正面評論：4000
負面評論：7987


In [ ]:
pd_all.sample(10)

,label,review
11451,0,烤腸忘給了
3097,1,味道不錯，快遞師傅贊贊噠
604,1,比預定時間快了20分鐘～很好～
8397,0,今日菜有些鹹
2414,1,上次說錯了，培根味道一般、美式密肉的好吃，真的鮮榨果汁！
2396,1,不錯，下次還會再買的
715,1,"送餐超快,面不坨,味道好"
5822,0,慢，早餐吃成午餐
1984,1,很好吃，謝謝
11109,0,速度慢，1個多小時才到，而且，我的面竟然沒有湯


#### 切 training data 及 testing data

In [ ]:
# 將要訓練的句子存起來
def save_sentence(filepath, sent_list):
    f = open(filepath, 'w', encoding='utf-8')
    for sent in sent_list:
        f.write(sent + '\n')
    f.close()

# 分割 train_data 和 test_data
def split_data(filename):

    fp = open(filename, 'r', encoding='utf-8')
    line = fp.readline() # 第一行是label, review
    line = fp.readline()

    train_sent_num = 3000
    test_sent_num = 1000
    # 計算目前資料筆數
    train_positive_num = 0 
    train_negative_num = 0
    test_positive_num = 0
    test_negative_num = 0
    train_data = []
    test_data = []

    # 用 while 逐行讀取檔案內容，直至檔案結尾
    while line:
        sent = ''
        sent = line.replace('\n', '')

        if line[:2] == '1,':
            if train_positive_num < train_sent_num:
                train_data.append(sent)
                train_positive_num += 1
            elif test_positive_num < test_sent_num:
                test_data.append(sent)
                test_positive_num += 1
        else:
            if train_negative_num < train_sent_num:
                train_data.append(sent)
                train_negative_num += 1
            elif test_negative_num < test_sent_num:
                test_data.append(sent)
                test_negative_num += 1
        
        line = fp.readline()
    
    fp.close()

    save_sentence('./train_data.txt', train_data)
    save_sentence('./test_data.txt', test_data)

In [ ]:
split_data('waimai_10k_zh_tw.csv')

In [ ]:
!ls

'BERT on 外賣資料集.ipynb'   train_data.txt
 test_data.txt		     waimai_10k_zh_tw.csv


#### 載入相關套件

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.2MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c8102c8c3307024dbb8c5949da20cbed2b578cd5a2557d46811f7f3c9ad56150
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import (AdamW, BertForSequenceClassification, BertTokenizerFast,
                          Trainer, TrainingArguments)

#### 讀取資料（整理出資料中的 sentence、label）

In [ ]:
def read_waimai(path):
    with open(path,'r',encoding='utf-8') as f:
        data = f.read()
    LS_pairs = data.split("\n")
    
    texts = []
    labels = []
    for LS_pair in LS_pairs:
        if LS_pair != "":
            try:
                L = LS_pair[:1]
                S = LS_pair[2:]
                texts.append(S)
                labels.append(int(L))
            except:
                continue
    return texts, labels

train_texts, train_labels = read_waimai('./train_data.txt')
test_texts, test_labels = read_waimai('./test_data.txt')

In [ ]:
print(train_texts[0], train_labels[0])

很快，好吃，味道足，量大 1


#### Tokenize（將資料轉換成 token id 、tpye_id 與 attention_mask）
`truncation=True` 和 `padding=True`

確保輸入不超過模型的最大限制以及讓每一筆資料長度都一樣

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
print(train_encodings[0].ids)
print(tokenizer.decode(train_encodings[0].ids))

print(train_encodings[0].type_ids)
print(train_encodings[0].attention_mask)

[101, 2523, 2571, 8024, 1962, 1391, 8024, 1456, 6887, 6639, 8024, 7030, 1920, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### 定義 Dataset，並轉換成 tensor 格式

In [ ]:
class WaimaiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = WaimaiDataset(train_encodings, train_labels)
test_dataset = WaimaiDataset(test_encodings, test_labels)

In [ ]:
print(train_dataset[0])

{'input_ids': tensor([ 101, 2523, 2571, 8024, 1962, 1391, 8024, 1456, 6887, 6639, 8024, 7030,
        1920,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 

#### Train

In [ ]:
# 設定訓練參數
training_args = TrainingArguments(
    output_dir='./trained_model',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

# 使用 bert-base-chinese 預訓練模型
model = BertForSequenceClassification.from_pretrained("bert-base-chinese")

# 將參數與資料丟入 trainer 
trainer = Trainer(
    model=model,                 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
%%time
trainer.train()

Step,Training Loss
500,0.410871
1000,0.346714
1500,0.300215


CPU times: user 10min 23s, sys: 7min 21s, total: 17min 44s
Wall time: 17min 57s


TrainOutput(global_step=1500, training_loss=0.3526003011067708)

In [ ]:
# 儲存模型
trainer.save_model("./trained_model")

#### Inference

In [ ]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
config = BertConfig.from_pretrained("./trained_model/config.json") 
model = BertForSequenceClassification.from_pretrained("./trained_model/pytorch_model.bin", config=config)

sentence = "這間餐廳的東西很好吃。"

inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)
predicts = outputs[0]
max_val = torch.max(predicts)
predict_label = (predicts == max_val).nonzero().numpy()[0][1]

if str(predict_label) == '1':
    print('正面')
else:
    print('負面')

正面
